# pytorch 모델 만들기
- pytorch 모델의 기초
- 모델의 레이어 및 기타 컴포넌트를 정의하는 __init_() 메서드와 계산이 실행되는 forward() 메서드가 있습니다. 모델 또는 모델 하위 모듈을 인쇄하여 해당 구조에 대해 알아볼 수 있습니다.

In [4]:
import torch

class TinyModel(torch.nn.Module):

  def __init__(self):
    super(TinyModel, self).__init__()

    self.linear1 = torch.nn.Linear(100, 200)
    self.activation = torch.nn.ReLU()
    self.linear2 = torch.nn.Linear(200, 10)
    self.softmax = torch.nn.Softmax()


  def forward(self, x):
    x = self.linear1(x)
    x = self.activation(x)
    x = self.linear2(x)
    x = self.softmax(x)
    return x

tinymodel = TinyModel()

print("The model:")
print(tinymodel)

print('\n\nJust one layer:')
print(tinymodel.linear2)

print('\n\nModel params:')
for param in tinymodel.parameters():
  print(param)

print('\n\nLayer params:')
for param in tinymodel.linear2.parameters(): 
  print(param)

The model:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Just one layer:
Linear(in_features=200, out_features=10, bias=True)


Model params:
Parameter containing:
tensor([[-7.2880e-02, -2.9889e-02, -3.6837e-04,  ..., -4.3601e-02,
         -7.4609e-02, -4.0679e-02],
        [ 5.8475e-03,  3.6911e-02,  4.7242e-02,  ...,  4.7107e-02,
          3.3355e-03, -3.9959e-05],
        [-4.2303e-02,  9.8925e-02,  3.2097e-02,  ..., -7.2476e-02,
          2.4686e-02,  5.9391e-02],
        ...,
        [-7.8780e-02, -4.2372e-02,  4.5320e-02,  ..., -9.7884e-02,
          2.4030e-02,  3.6723e-02],
        [ 1.6981e-02,  9.3357e-03,  6.4435e-02,  ...,  4.0301e-02,
          8.1140e-02, -4.9925e-02],
        [ 4.6042e-02,  5.2944e-02, -7.8302e-02,  ..., -3.1849e-02,
         -8.8971e-02, -6.0014e-02]], requires_grad=True)
Parameter containing:
tensor([-

# Layer Types


## Linear Layers
- 신경망에 기초가 되는 선형 or 완전 결합층
- 모든 입력이 레이어의 모든 출력에 영향을 미치는 레이어이며 레이어의 가중치에 의해 지정된 정도까지 영향을 줍니다

In [5]:
lin = torch.nn.Linear(3, 2)
x = torch.rand(1, 3)
print('Input:')
print(x)

print('\n\nWeight and Bias parameters:')
for param in lin.parameters():
  print(param)

y = lin(x)
print('\n\nOutput:')
print(y)

Input:
tensor([[0.5391, 0.5677, 0.6381]])


Weight and Bias parameters:
Parameter containing:
tensor([[ 0.1799,  0.1469, -0.0719],
        [-0.2450,  0.0031, -0.3007]], requires_grad=True)
Parameter containing:
tensor([ 0.4159, -0.1698], requires_grad=True)


Output:
tensor([[ 0.5504, -0.4919]], grad_fn=<AddmmBackward0>)


## Convolutional Layers
- LeNet5는 1x32x32 블랙화이트 이미지가 들어감 -> input channel 1이다, color -> 3

In [7]:
import torch.functional as F

class LeNet(torch.nn.Module):

  def __init__(self):
    super(LeNet, self).__init__()
    self.conv1 = torch.nn.Conv2d(1, 6, 5) # (in_channel, out_channel, kerenl_size)
    self.conv2 = torch.nn.Conv2d(6, 16, 3)
    # an affine operation : y = Wx + b
    self.fc1 = torch.nn.Linear(16 * 6 * 6, 120) # 6 * 6 from image dimension
    self.fc2 = torch.nn.Linear(120, 84)
    self.fc3 = torch.nn.Linear(84, 10)


  def forward(self, x):
    # Max pooling over a (2, 2) window
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_float_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def num_flat_features(self, x):
    size = x.size()[1:]
    num_features = 1
    for s in size:
      num_features *= s
    return num_features

In [8]:
model = LeNet()

In [9]:
print(model)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# Recurrent Layers
- 연속적인 데이터, 시계열 데이터에 사용 
- 원핫 벡터의 인덱스로 표현된 단어가 input 값
- embedding_dim의 차원에 단어가 맵핑됨
- LSTM에서는 단어를 반복하여 임베딩 한 후 output vector가 hidden_dim 길이에 맞춰 생성
- 마지막에는 log_softmax()에 들어감

In [10]:
class LSTMTagger(torch.nn.Module):

  def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
    super(LSTMTagger, self).__init__()
    self.hidden_dim = hidden_dim

    self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

    # The LSTM takes word embedding as inputs, and outputs hidden states
    self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

    self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

  def forward(self, sentence):
    embeds = self.word_embeddings(sentence)
    lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
    tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
    tag_scores = F.log_softmax(tag_space, dim = 1)
    return tag_scores

## 다른 레이어 및 함수

In [18]:
my_tensor = torch.rand(1, 6, 6)
print(my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3)
print(maxpool_layer(my_tensor))

maxpool_layer2 = torch.nn.MaxPool2d(2)
print(maxpool_layer2(my_tensor))

tensor([[[0.7607, 0.6327, 0.4919, 0.0871, 0.4490, 0.9333],
         [0.1627, 0.4619, 0.8735, 0.5496, 0.7042, 0.1770],
         [0.4394, 0.1749, 0.3668, 0.2415, 0.4574, 0.2301],
         [0.7467, 0.3982, 0.6218, 0.6669, 0.0118, 0.5965],
         [0.7691, 0.8661, 0.2570, 0.3681, 0.8430, 0.5603],
         [0.0172, 0.5268, 0.0133, 0.2996, 0.7011, 0.6116]]])
tensor([[[0.8735, 0.9333],
         [0.8661, 0.8430]]])
tensor([[[0.7607, 0.8735, 0.9333],
         [0.7467, 0.6669, 0.5965],
         [0.8661, 0.3681, 0.8430]]])


In [19]:
# Normalization layers를 통하여 값이 튀는 것을 방지함
my_tensor = torch.rand(1, 4, 4) * 20 + 5
print(my_tensor)

print(my_tensor.mean())

norm_layer = torch.nn.BatchNorm1d(4)
normed_tensor = norm_layer(my_tensor)
print(normed_tensor)

print(normed_tensor.mean())

tensor([[[23.8677, 22.5560, 17.9639, 11.1986],
         [21.5940, 12.2505, 13.9888, 12.7835],
         [14.3400, 22.7724, 21.7758, 14.7495],
         [ 8.8113, 12.7816,  5.7468, 24.2140]]])
tensor(16.3372)
tensor([[[ 1.0031,  0.7384, -0.1882, -1.5534],
         [ 1.7077, -0.7700, -0.3090, -0.6287],
         [-1.0479,  1.1235,  0.8669, -0.9424],
         [-0.5826, -0.0153, -1.0205,  1.6183]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(-1.8254e-07, grad_fn=<MeanBackward0>)


In [20]:
# Dropout layers
my_tensor = torch.rand(1, 4, 4)
dropout = torch.nn.Dropout(p=0.4)
print(dropout(my_tensor))
print(dropout(my_tensor))

tensor([[[1.2890, 0.0000, 0.8293, 0.9484],
         [0.0851, 0.0000, 0.4085, 0.7532],
         [1.2670, 1.1030, 0.0000, 0.0000],
         [0.2854, 0.9911, 0.0000, 1.0306]]])
tensor([[[0.0000, 0.0000, 0.8293, 0.0000],
         [0.0851, 0.0000, 0.4085, 0.7532],
         [1.2670, 1.1030, 0.7541, 0.8681],
         [0.2854, 0.0000, 0.0000, 0.0000]]])
